In [26]:
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.svm import SVC

### Load dataframe and separate it into train and test

In [18]:
all_df = pd.read_csv("participants_dataset.csv", index_col=0)
all_df.head()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,label
ID,,,,,,,,,,,,,,,,,,,,,
452,steven lux,steven,lux,2013-01-05,Male,1953-06-15,62,Greater than 45,Caucasian,0,...,1,Low,2013-01-05,2014-03-25,2014-03-26,0,2,444,0,0.0
573,andre small,andre,small,2013-04-03,Male,1987-10-01,28,25 - 45,African-American,0,...,3,Low,2013-04-03,2013-04-02,2013-04-04,1,1,1094,0,0.0
4952,willie gray,willie,gray,2014-10-31,Male,1959-01-12,57,Greater than 45,African-American,0,...,2,Low,2014-10-31,2014-10-31,2014-10-31,8,0,265,1,1.0
3521,nickson marcellus,nickson,marcellus,2014-01-23,Male,1996-07-11,19,Less than 25,African-American,0,...,10,High,2014-01-23,2016-01-05,2016-02-26,1,0,711,1,1.0
369,patria barnes,patria,barnes,2013-12-09,Female,1978-06-06,37,25 - 45,Other,0,...,1,Low,2013-12-09,2013-12-08,2013-12-09,0,0,844,0,0.0


In [19]:
all_df.tail()

,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,juv_fel_count,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,label
ID,,,,,,,,,,,,,,,,,,,,,
2361,cindy stillman,cindy,stillman,2013-08-11,Female,1967-01-06,49,Greater than 45,Caucasian,0,...,1,Low,2013-08-11,2013-08-10,2013-08-22,2,11,245,1,NaN
1865,mario brown,mario,brown,2013-11-20,Male,1979-11-24,36,25 - 45,Caucasian,0,...,1,Low,2013-11-20,2013-11-19,2013-12-14,0,24,863,0,NaN
4086,andre barrington,andre,barrington,2014-01-24,Male,1989-06-12,26,25 - 45,African-American,0,...,10,High,2014-01-24,2014-05-04,2014-05-09,13,7,100,0,NaN
4569,tyson harper,tyson,harper,2013-08-29,Male,1978-10-02,37,25 - 45,Caucasian,0,...,1,Low,2013-08-29,2014-07-18,2014-12-02,5,0,323,0,NaN
8403,shawn smith,shawn,smith,2014-09-23,Male,1996-07-10,19,Less than 25,African-American,0,...,7,Medium,2014-09-23,2014-10-16,2014-12-12,0,0,23,1,NaN


In [21]:
all_df = pd.concat([all_df.dropna(axis=1), all_df["label"]], axis=1)

In [23]:
sample_data = all_df.iloc[0]
for i in range(0, len(sample_data)):
    if isinstance(sample_data[i], str):
        mapper = {value:count for count, value in enumerate(all_df.iloc[:,i].value_counts().index)}
        all_df.iloc[:,i] = all_df.iloc[:,i].replace(mapper)

In [24]:
train_df = all_df[np.isnan(all_df["label"])==False]
test_df = all_df[np.isnan(all_df["label"])]

### Define and train model on train_df

In [25]:
clf = SVC()
clf.fit(train_df.iloc[:,:-1], train_df.iloc[:,-1])

SVC()

### Score model based on training data

In [31]:
y_pred = clf.predict(train_df.iloc[:, :-1])
cr = classification_report(train_df.iloc[:, -1], y_pred)
print(cr)

              precision    recall  f1-score   support

         0.0       0.93      0.85      0.89      3171
         1.0       0.83      0.92      0.88      2600

    accuracy                           0.88      5771
   macro avg       0.88      0.89      0.88      5771
weighted avg       0.89      0.88      0.88      5771



### Apply model on test data

In [32]:
y_pred = clf.predict(test_df.iloc[:, :-1])

In [33]:
new_test_df = test_df.copy(deep=True)
new_test_df["label"] = y_pred

In [35]:
new_test_df.to_csv("Recidivism_submission.csv")